In [0]:
from pyspark.sql import functions as F, Window

CATALOG = "energy_usage_data_platform"
GOLD_SCHEMA = f"{CATALOG}.gold"
SILVER_SCHEMA = f"{CATALOG}.silver"

In [0]:
daily = spark.table(f"{GOLD_SCHEMA}.daily_region_usage")

In [0]:
# Compute solar quantiles per year + region_id
quantiles = (
    daily
    .filter(F.col("avg_daily_ghi_w_m2").isNotNull())
    .groupBy("year", "region_id")
    .agg(
        F.expr("percentile_approx(avg_daily_ghi_w_m2, array(0.33, 0.66))").alias("q")
    )
    .select(
        "year",
        "region_id",
        F.col("q")[0].alias("q33"),
        F.col("q")[1].alias("q66"),
    )
)

daily_with_q = daily.join(quantiles, on=["year", "region_id"], how="left")

In [0]:
# Bucket days into low / medium / high solar
daily_bucketed = (
    daily_with_q
    .withColumn(
        "solar_bucket",
        F.when(F.col("avg_daily_ghi_w_m2").isNull(), F.lit("unknown"))
         .when(F.col("avg_daily_ghi_w_m2") <= F.col("q33"), "low")
         .when(F.col("avg_daily_ghi_w_m2") <= F.col("q66"), "medium")
         .otherwise("high")
    )
)

In [0]:

# Aggregate by bucket
solar_impact = (
    daily_bucketed
    .groupBy("year", "region_id", "substation_id", "solar_bucket")
    .agg(
        F.count("*").alias("num_days"),
        F.avg("daily_kwh_total").alias("avg_daily_kwh"),
        F.avg("daily_kw_peak").alias("avg_daily_kw_peak"),
        F.avg("avg_daily_temp_c").alias("avg_daily_temp_c"),
        F.avg("avg_daily_ghi_w_m2").alias("avg_daily_ghi_w_m2"),
    )
)

In [0]:
(
    solar_impact
    .write
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(f"{GOLD_SCHEMA}.solar_impact_on_load")
)
display(spark.table(f"{GOLD_SCHEMA}.solar_impact_on_load").limit(10))